<a href="https://colab.research.google.com/github/alekhyeah/Text_Summarization_Infosys_Internship_Oct2024/blob/AlekhyaReddy/Week%204-5%20NLP_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.6 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=66128573e4dd37a51feaf79a24a6ef08bc913079c0b20350500589d96dd1b9aa
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f0deb253abdd0b09714686382ba3c145b8dcfaf5c01534b7b839e890838fb890
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
pip install langchain[google]

In [ ]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain langchain-google-genai langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
import gradio as gr
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from transformers import T5ForConditionalGeneration, T5Tokenizer, BartForConditionalGeneration, BartTokenizer

In [ ]:
# Download nltk data
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Functions for Extractive Summarization Methods
def frequency_summarize(text, max_words):
    stopwords1 = set(stopwords.words("english"))
    words = word_tokenize(text)
    freqTable = {word: words.count(word) for word in set(words) if word.lower() not in stopwords1}
    sentences = sent_tokenize(text)
    sentenceValue = {}
    for sentence in sentences:
        sentenceValue[sentence] = sum(freqTable.get(word.lower(), 0) for word in word_tokenize(sentence))
    sorted_sentences = sorted(sentenceValue.items(), key=lambda item: item[1], reverse=True)
    summary = ' '.join(sentence for sentence, score in sorted_sentences[:3])
    if len(summary.split()) > max_words:
        summary = ' '.join(summary.split()[:max_words])
    return summary

def lsa_summarize(text, max_words):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = ' '.join(str(sentence) for sentence in summarizer(parser.document, 3))
    return summary if len(summary.split()) <= max_words else ' '.join(summary.split()[:max_words])

def lexrank_summarize(text, max_words):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = ' '.join(str(sentence) for sentence in summarizer(parser.document, 3))
    return summary if len(summary.split()) <= max_words else ' '.join(summary.split()[:max_words])

def luhn_summarize(text, max_words):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LuhnSummarizer()
    summary = ' '.join(str(sentence) for sentence in summarizer(parser.document, 3))
    return summary if len(summary.split()) <= max_words else ' '.join(summary.split()[:max_words])

In [ ]:
# Abstractive Summarization Methods
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

def t5_summarize(text, max_words):
    inputs = t5_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs, max_length=max_words, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def bart_summarize(text, max_words):
    inputs = bart_tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs, max_length=max_words, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# LLM Summarizer
def load_llm(model="gemini-1.5-pro"):
    return ChatGoogleGenerativeAI(model=model, temperature=0, max_tokens=None, timeout=None, max_retries=2)

def get_prompt_template():
    return ChatPromptTemplate.from_messages([("system", "Summarize this in {num_words} words:\n\n"), ("human", "{context}")])

def llm_summarize(text, model="gemini-1.5-pro", num_words=50):
    llm = load_llm(model)
    prompt = get_prompt_template()
    chain = prompt | llm
    result = chain.invoke({"context": text, "num_words": num_words})
    return result.content

In [ ]:
import gradio as gr

def word_count(text):
    return len(text.split())

with gr.Blocks(title="Summarizer App") as demo:
    gr.Markdown("# Summarizer App")

    with gr.Tabs():
        # Extractive summarization tab
        with gr.TabItem("Extractive Summarization"):
            extractive_method_choice = gr.Dropdown(choices=["Frequency", "TF-IDF", "TextRank"], label="Select Extractive Method")
            extractive_max_words = gr.Number(label="Max Words", value=50)

            with gr.Row():
                # Left column for input text
                with gr.Column(scale=1):
                    extractive_input_text = gr.Textbox(label="Input Text", lines=10, placeholder="Enter text to summarize")

                # Right column for output text and word counts
                with gr.Column(scale=1):
                    extractive_output_text = gr.Textbox(label="Extractive Summary", placeholder="Summary will appear here")
                    input_word_count = gr.Textbox(label="Input Word Count", value="0", interactive=False)
                    output_word_count = gr.Textbox(label="Output Word Count", value="0", interactive=False)

            def extractive_summarize(text, method, max_words):
                if method == "Frequency":
                    summary = frequency_summarize(text, max_words)
                elif method == "TF-IDF":
                    summary = lsa_summarize(text, max_words)
                elif method == "TextRank":
                    summary = lexrank_summarize(text, max_words)
                else:
                    return "Invalid method selected.", 0

                return summary, word_count(text), word_count(summary)

            extractive_generate_button = gr.Button("Generate Extractive Summary")
            extractive_generate_button.click(
                fn=extractive_summarize,
                inputs=[extractive_input_text, extractive_method_choice, extractive_max_words],
                outputs=[extractive_output_text, input_word_count, output_word_count]
            )

        # Abstractive summarization tab
        with gr.TabItem("Abstractive Summarization"):
            abstractive_method_choice = gr.Dropdown(choices=["T5", "BART"], label="Select Abstractive Method")
            abstractive_max_words = gr.Number(label="Max Words", value=50)

            with gr.Row():
                # Left column for input text
                with gr.Column(scale=1):
                    abstractive_input_text = gr.Textbox(label="Input Text", lines=10, placeholder="Enter text to summarize")

                # Right column for output text and word counts
                with gr.Column(scale=1):
                    abstractive_output_text = gr.Textbox(label="Abstractive Summary", placeholder="Summary will appear here")
                    input_word_count_abstractive = gr.Textbox(label="Input Word Count", value="0", interactive=False)
                    output_word_count_abstractive = gr.Textbox(label="Output Word Count", value="0", interactive=False)

            def abstractive_summarize(text, method, max_words):
                if method == "T5":
                    summary = t5_summarize(text, max_words)
                elif method == "BART":
                    summary = bart_summarize(text, max_words)
                #elif method == "LLM":
                    #summary = llm_summarize(text, model="gemini-1.5-pro", num_words=max_words)
                else:
                    return "Invalid method selected.", 0

                return summary, word_count(text), word_count(summary)

            abstractive_generate_button = gr.Button("Generate Abstractive Summary")
            abstractive_generate_button.click(
                fn=abstractive_summarize,
                inputs=[abstractive_input_text, abstractive_method_choice, abstractive_max_words],
                outputs=[abstractive_output_text, input_word_count_abstractive, output_word_count_abstractive]
            )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b7e0aa61b4fcb7e30f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
